In [242]:
from manifold3d import Manifold, CrossSection, set_circular_segments
import math
import numpy as np
set_circular_segments(32)

In [282]:
def generate_bit(type, diameter, length=50e-3):
    if type == "endmill":
        return Manifold.cylinder(length, diameter/2)
    elif type == "ballnose":
        shaft = Manifold.cylinder(length-diameter/2, diameter/2)
        bit = Manifold.sphere(diameter/2)
        return (shaft + bit).translate([0,0,diameter/2])
    elif type == "drill":
        conelen = 0.5*diameter/math.tan(math.radians(59))
        shaft = Manifold.cylinder(length-conelen, diameter/2)
        bit = Manifold.cylinder(conelen, diameter/2, 0).rotate(np.array([180, 0, 0]))
        return (shaft + bit).translate([0,0,conelen])

In [327]:
loc1 = np.array([0, 0, 0])
loc2 = np.array([10e-3, 10e-3, 5e-3])
tool = "ballnose"
dcyl = 0 if tool == "endmill" else (2e-3 if tool == "ballnose" else 0.5*4e-3/math.tan(math.radians(59)))
mesh = generate_bit(tool, 4e-3).translate(loc1)

meshz = Manifold.compose([mesh]).translate(-loc1)
mesh2 = Manifold.compose([mesh]).translate(loc2)
delta = loc2 - loc1
dxy = (delta[0]**2 + delta[1]**2)**0.5
dxyz = (delta[0]**2 + delta[1]**2 + delta[2]**2)**0.5
phi = math.atan2(delta[1], delta[0])+np.pi
theta = math.atan2(dxy, delta[2])
cutarea = Manifold()
if dxy > 1e-9:
    rectrod = CrossSection.square([4e-3, (50e-3-dcyl)*np.sin(theta)]).translate([-2e-3, 0])
    meshz = meshz
    cutarea += Manifold.extrude(rectrod, dxyz)\
        .warp(lambda v: [v[0], v[1], v[2]+v[1]/math.tan(theta)])\
        .rotate(np.array([np.rad2deg(theta), 0, -np.rad2deg(phi)]))\
        .translate([0,0,dcyl])
if delta[2] > 1e-9:
    circ = CrossSection.circle(4e-3/2)
    cutarea += Manifold.extrude(circ, delta[2])\
        .warp(lambda v: [v[0], v[1]-v[2]*math.tan(theta), v[2]])\
        .rotate([-np.rad2deg(theta),0,0])\
        .rotate(np.array([np.rad2deg(theta), 0, -np.rad2deg(phi)]))\
        .translate([0,0,dcyl])
if tool == "ballnose":
    circ = CrossSection.circle(4e-3/2).translate([0,4e-3/2*np.sin(theta)])
    cutarea += Manifold.extrude(circ, delta[2])\
        .translate([0,0,4e-3/2*np.cos(theta)])\
        .rotate(np.array([np.rad2deg(theta), 0, -np.rad2deg(phi)]))
cutarea += meshz + mesh2

In [328]:
import polyscope as ps

ps.init()
ps.set_up_dir("z_up")
ps.set_front_dir("neg_y_front")
ps.register_surface_mesh("meshz", meshz.to_mesh().vert_properties[:, :3], meshz.to_mesh().tri_verts)
ps.register_surface_mesh("mesh2", mesh2.to_mesh().vert_properties[:, :3], mesh2.to_mesh().tri_verts)
ps.register_surface_mesh("mesh3", cutarea.to_mesh().vert_properties[:, :3], cutarea.to_mesh().tri_verts)
ps.show()